In [ ]:
import time
import datetime
import sqlite3
import pandas as pd
import matplotlib as plt
import seaborn as sns  # für das Styling der Ausgabe https://pandas.pydata.org/pandas-docs/stable/style.html#Builtin-Styles


# Ziel: Wie viele Songs wurden am Tag gespielt. Am besten für das ganze Jahr

# Erstmal das heutige Datum feststellen
today = datetime.datetime.today()
start = "2019-01-01T00:01"

# Daten aus Datenbank laden
connection = sqlite3.connect('../db.sqlite3')
df = pd.read_sql_query("""SELECT bayern3.id, datum_zeit, interpret, titel, label FROM bayern3 
                        INNER JOIN songs ON bayern3.song=songs.id""", connection, index_col='id')
connection.close()

# Separaten DataFrame für den Vergleich
vergleich = pd.DataFrame()
vergleichlabel = pd.DataFrame()
# Eine Spalte je Kalenderwoche, titel als Index

for i in range(erstewoche+1, weeknr+1):
    vergleich['KW'+str(i)] = df[df['woche'] == i].groupby('titel').count().sort_values(by='datum_zeit', ascending=False)['datum_zeit']
    vergleichlabel['KW'+str(i)] = df[df['woche'] == i].groupby('label').count().sort_values(by='datum_zeit', ascending=False)['datum_zeit']

    
# Alle Zeilen droppen, die NaN enthalten (liegt vor, wenn der Titel in einer Woche nicht gespielt wurde)
vergleich = vergleich.dropna(how='any')
vergleichlabel = vergleichlabel.dropna(how='any')
# Alternativ mit 0 ersetzen
# vergleich = vergleich.fillna(value=0)

# eine Zeile löschen, z.B. NaN im Label
vergleichlabel = vergleichlabel.drop(["NaN"])
####################################
### funktionierende Auswertungen ###
####################################

tabelle = vergleichlabel.head(10)
#dfweek = pd.DataFrame(vergleich, columns=["KW12"]).sort_values(by='KW12', ascending=False)
#dftitel = pd.DataFrame(vergleich, index=["Crazy"])
#dftitel
#dfweek
cm = sns.light_palette("green", as_cmap=True)
ergebnis = tabelle.style.background_gradient(cmap=cm)
ergebnis